In [67]:
import pandas as pd
import numpy as np
from pymoo.algorithms.unsga3 import UNSGA3
from pymoo.optimize import minimize
import pickle

from combined_problem import CombinedProblem, CombinedProblemDownSelect
from notebooks.optimization_problems.constraints import Requirements

In [68]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "telesat_post_processed.h5"

row_limit = -1

store_500km = pd.HDFStore(file_path)

instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)
instances_df = instances_500km_df

In [69]:
from notebooks.optimization_problems.design_vector import design_vector_default_scm, SystemParameters

sys_param = SystemParameters()
sys_param.margin_dB = 3.0
sys_param.B_Hz_array = np.array([0.1, 0.5, 1, 10, 50, 100, 200, 300]) * 1e6
sys_param.Gtx_dBi_bounds = (3., 30.)
sys_param.Ptx_dBm_bounds = (20., 43.)

modcods_df = pd.read_pickle('../dvbs2.pkl')

sys_param.EsN0_req_dB_array = modcods_df[['isend', 'isend_200MHz', 'isend_300MHz']].to_numpy()
sys_param.eta_bitsym_array = modcods_df[['eta', 'eta_200MHz', 'eta_300MHz']].to_numpy()
sys_param.eta_maee_array = modcods_df[['maee_12', 'maee_12_200MHz', 'maee_12_300MHz']].to_numpy()

requirements = Requirements()
requirements.min_throughput = 120e9
requirements.max_latency = 60.0 * 60.0 * 1.5

In [70]:
from pymoo.factory import get_reference_directions, get_visualization, get_termination

cases = []
case_100 = {
    'label': 'n_gen_1600_n_pop_100',
    'problem': CombinedProblem(instances_df, sys_param, requirements=requirements, f_mask=np.array([0, 2, 3])),
    'n_gen': 1600,
    'ref_dirs': get_reference_directions("das-dennis", 3, n_partitions=12),
}
cases.append(case_100)

case_400 = {
    'label': 'n_gen_1600_n_pop_400',
    'problem': CombinedProblem(instances_df, sys_param, requirements=requirements, f_mask=np.array([0, 2, 3])),
    'n_gen': 1600,
    'ref_dirs': get_reference_directions("das-dennis", 3, n_partitions=25),
}
cases.append(case_400)

case_ds = {
    'label': 'n_gen_1600_n_pop_100_ds',
    'problem': CombinedProblemDownSelect(instances_df, sys_param, requirements=requirements, f_mask=np.array([0, 2, 3])),
    'n_gen': 1600,
    'ref_dirs': get_reference_directions("das-dennis", 3, n_partitions=12),
}
cases.append(case_ds)

case_400_ds = {
    'label': 'n_gen_1600_n_pop_400_ds',
    'problem': CombinedProblemDownSelect(instances_df, sys_param, requirements=requirements, f_mask=np.array([0, 2, 3])),
    'n_gen': 1600,
    'ref_dirs': get_reference_directions("das-dennis", 3, n_partitions=25),
}
cases.append(case_400_ds)

In [71]:
case = cases[3]

problem = case['problem']
sampling, crossover, mutation = design_vector_default_scm(problem.x_length, problem.x_indices)

In [72]:
settings = {}

# U-NSGA-III
ref_dirs = case['ref_dirs']
print(len(ref_dirs))
settings['UNSGA-III'] = {
    'label': 'unsga3',
    'algorithm': UNSGA3(
        pop_size=int(np.ceil(len(ref_dirs) / 100) * 100),
        n_offsprings=int(np.ceil(len(ref_dirs) / 100) * 100 / 4),
        sampling=sampling,#InitialSampling(),
        crossover=crossover,
        mutation=mutation,
        ref_dirs=ref_dirs,
        eliminate_duplicates=True,
    ),
    'termination': ('n_gen', case['n_gen'])
}

setting = settings['UNSGA-III']

termination = setting['termination']
algorithm = setting['algorithm']

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               #save_history=True,
               verbose=True,
               copy_algorithm=False,
               )

print('Processes:', res.exec_time)
print("Best solution found: %s" % res.X)

file_name = 'down_selection_%s.pkl' % (case['label'])

pickle.dump((problem, setting, res), open(file_name, 'wb'))

351
    1 |     400 |  0.00000E+00 |  0.482157459 |      29 |  1.919642857 |        ideal
    2 |     500 |  0.00000E+00 |  0.278919586 |      31 |  0.036352637 |            f
    3 |     600 |  0.00000E+00 |  0.120550776 |      32 |  0.042915115 |        ideal
    4 |     700 |  0.00000E+00 |  0.002832105 |      37 |  0.068056249 |        ideal
    5 |     800 |  0.00000E+00 |  0.00000E+00 |      35 |  0.247058046 |        ideal
    6 |     900 |  0.00000E+00 |  0.00000E+00 |      35 |  0.020969682 |            f
    7 |    1000 |  0.00000E+00 |  0.00000E+00 |      37 |  0.019451020 |        ideal
    8 |    1100 |  0.00000E+00 |  0.00000E+00 |      38 |  0.034976100 |        ideal
    9 |    1200 |  0.00000E+00 |  0.00000E+00 |      36 |  0.018767268 |            f
   10 |    1300 |  0.00000E+00 |  0.00000E+00 |      35 |  0.014786018 |            f
   11 |    1400 |  0.00000E+00 |  0.00000E+00 |      33 |  0.011052379 |        ideal
   12 |    1500 |  0.00000E+00 |  0.00000E+00 |   

In [73]:
from beepy import beep
for i in range(3):
    beep(sound=5)